In [35]:
%cd ..

c:\Users\joelw\OneDrive\Documents\Projects\Programming\rust-in-python


In [40]:
%pwd

'c:\\Users\\joelw\\OneDrive\\Documents\\Projects\\Programming\\rust-in-python'

In [42]:
import rust_mod
import timeit
import pandas as pd
import plotly.express as px
from sklearn import linear_model
import numpy as np
import plotly.graph_objects as go
from pysrc import funs as pyfun
import importlib
importlib.reload(pyfun)
importlib.reload(rust_mod)


<module 'rust_mod' from 'c:\\Users\\joelw\\OneDrive\\Documents\\Projects\\Programming\\rust-in-python\\rust_mod\\.env\\lib\\site-packages\\rust_mod\\__init__.py'>

In [38]:
a = timeit.timeit(f"pyfun.get_fibonacci(150)", setup="from pysrc import funs as pyfun")
b = timeit.timeit(f"rust_mod.get_fibonacci(150)", setup="import rust_mod")

In [54]:
names = ['Python', 'Rust Develop']
timeit_fun = {'python': "get_fibonacci", 
               'rust': "get_fibonacci"}
timeit_setup = {'python': "from pysrc.funs import get_fibonacci", 
               'rust': "from rust_mod import get_fibonacci"}      
test_input = list(range(0, 151, 50))

In [55]:
values = {name: [] for name in names}
for i in test_input:
    for name in names:
        v = timeit.timeit(f"{timeit_fun['python']}({i})", setup=f"{timeit_setup['python']}")
        values[name].append(v)
        
    if i % ((len(test_input)//10+1)) == 0:
        print(i, 'out of ', test_input[-1])


0 out of  [0, 50, 100, 150]
50 out of  [0, 50, 100, 150]
100 out of  [0, 50, 100, 150]
150 out of  [0, 50, 100, 150]


In [56]:
params = {}
for name in names:
    params[name] = {}
    X = np.array(test_input).reshape(len(test_input), 1)
    y = np.array(values[name])
    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X, y)
    
    v = ransac.predict(np.array([[0.],[1]]))
    params[name]['m'] = v[0]
    params[name]['k'] = v[1]-v[0]



In [57]:
colors = ['#55AA55','#AA5555', '#5555AA']
min_val = min([min(values[name]) for name in names])
max_val = max([max(values[name]) for name in names])
text_start = [min(test_input) + 2/10*(max(test_input) - min(test_input)),
              min_val + 9/10*(max_val - min_val)]
y_diff = 1/10*(max_val - min_val)

go.Figure(
    data = [
        go.Scatter(x = test_input, y = values[name],
        mode='markers',
        name=name,
        marker = {'color': colors[i]})
        for i, name in enumerate(names)] +
        [
            go.Scatter(x = [min(test_input), max(test_input)], y = [params[name]['m'], params[name]['m']+max(test_input)*params[name]['k']],
            mode='lines',
            showlegend = False,
            line = {'color': colors[i]} )
            for i, name in enumerate(names)] +
        [
            go.Scatter(x = [text_start[0]], y = [text_start[1]-i*y_diff],
                mode='text',
                showlegend = False,
                text = f"{name}: {params[name]['k']:0.2}x+{params[name]['m']:0.2}")
                for i, name in enumerate(names)]
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#55AA55'},
              'mode': 'markers',
              'name': 'Python',
              'type': 'scatter',
              'x': [0, 50, 100, 150],
              'y': [0.5296329999982845, 6.135857899993425, 12.258154300012393,
                    18.58452179998858]},
             {'marker': {'color': '#AA5555'},
              'mode': 'markers',
              'name': 'Rust Develop',
              'type': 'scatter',
              'x': [0, 50, 100, 150],
              'y': [0.45567869997466914, 6.925657500018133, 11.40260530001251,
                    16.658040100010112]},
             {'line': {'color': '#55AA55'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'x': [0, 150],
              'y': [0.33399732999969345, 18.420086169996647]},
             {'line': {'color': '#AA5555'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'x': [0, 150],
              'y': [0.8978905999887496, 16.82310020001896]},
             {'mode': 'text',
              'showlegend': False,
              'text': 'Python: 0.12x+0.33',
              'type': 'scatter',
              'x': [30.0],
              'y': [16.77163748998719]},
             {'mode': 'text',
              'showlegend': False,
              'text': 'Rust Develop: 0.11x+0.9',
              'type': 'scatter',
              'x': [30.0],
              'y': [14.958753179985798]}],
    'layout': {'template': '...'}
})

In [92]:
rm

1.285793263571968

In [93]:
rk

0.01960693866126295

In [90]:
pm

0.1599773742988182

In [91]:
pk

0.057946881339327294